In [1]:
from __future__ import print_function
import numpy as np
import torch
from torch.autograd import Variable
import pyro
from pyro.optim import Adam
from pyro.infer import SVI
import pyro.distributions as dist

In [25]:
pyro.clear_param_store()

In [33]:
data = []
for _ in range(12):
    data.append(Variable(torch.ones(1)))
for _ in range(4):
    data.append(Variable(torch.zeros(1)))

In [27]:
def model(data):
    alpha0 = Variable(torch.Tensor([10.0]))
    beta0  = Variable(torch.Tensor([10.0]))
    f = pyro.sample("latent_fairness", dist.beta, alpha0, beta0)
    for i in range(len(data)):
        pyro.observe("obs_{}".format(i), dist.bernoulli, data[i], f)

In [28]:
def guide(data):
    #why log 15?
    #How does defining alpha/beta in log space keep them non-negative? What if they'r [0, 1)
    log_alpha0 = Variable(torch.Tensor([np.log(15.0)]), requires_grad = True)
    log_beta0  = Variable(torch.Tensor([np.log(15.0)]), requires_grad = True)
    log_alpha  = pyro.param("log_alpha", log_alpha0)
    log_beta   = pyro.param("log_beta", log_beta0)
    alpha      = torch.exp(log_alpha0)
    beta       = torch.exp(log_beta0)
    pyro.sample("latent_fairness", dist.beta, alpha, beta)

In [29]:
adam_params = { 'lr':  0.0005, 'betas': (0.90, 0.999)}
optimizer   = Adam(adam_params)
svi         = SVI(model, guide, optimizer, loss="ELBO", num_particles=7)
n_steps = 4000
for step in range(n_steps):
    svi.step(data)
    if step % 100 == 0:
        print(".", end='')

..................................................

In [30]:
alpha = torch.exp(pyro.param("log_alpha")).data.numpy()[0]
beta  = torch.exp(pyro.param("log_beta")).data.numpy()[0]

inferred_mean = alpha / (alpha + beta)

factor = beta / (alpha  * (1 + alpha + beta))
inferred_std_dev = inferred_mean * np.sqrt(factor)

In [31]:
print("Inferred Mean: {}, Inferred Std Dev {}".format(inferred_mean, inferred_std_dev))

Inferred Mean: 0.5015009641647339, Inferred Std Dev 0.08980204865027393
